# Assume current path is project home directory, ~/PMG-Net

## 1. Train generator G first.

In [ ]:
# Note：如果报错 No module named util，在终端执行
export PYTHONPATH=$PYTHONPATH:~/PMG-Net/aa_nets:~/PMG-Net/stylegan3/dnnlib:~/PMG-Net/stylegan3/torch_utils
# 添加aanet和stylegan3所需的依赖库到环境变量中

In [ ]:
# # train diffusion-sgan2 (optional)
#% run diffusion-stylegan2/train.py --outdir=training-runs --data=datasets/invivo_disp_gt.npy  --gpus=1 --cfg stylegan2 --kimg 2000 --aug no --target 0.6 --noise_sd 0.05 --ts_dist priority --metrics none

# # train sgan2 with resume if existed (optional)
#% run stylegan3/train.py --outdir=training-runs --cfg=stylegan2 --data=datasets/invivo_disp_gt.npy --gpus=1 --batch=16 --gamma=10 --resume=/home/ubuntu/WS-YG/PMG-Net/final_models/invivo/optimizer_latest_sgan2.pkl

# train sgan3 without metrics evaluation
%run stylegan3/train.py --outdir=training-runs --cfg=stylegan3-t --data=datasets/invivo_disp_gt.npy --gpus=1 --batch=64 --gamma=8.2 --metrics=none

## 2. Generate label $w_{opt}^{train}$ for predictor P

In [ ]:
# Train w_opt for invivo training dataset
# check line237, change 'test' to 'train' when generate w_opt^{train}, to 'robust_test' when test robustness.
%run stylegan3/GNet_train.py --dataset=invivo --model=sgan3 --mode=train_w

## Train Predictor or Mapping

In [ ]:
# Train pmg for invivo
%run stylegan3/PMGNet_train.py --dataset=phantom --model=pmg --mode=test

## Others

In [ ]:
# Calculate FID, for example model file 001411.pkl and dataset Synth_invivo
%run stylegan3/calc_metrics.py --metrics=fid50k_full --network=~/WS-YG/PMG-Net/training-runs/network-snapshot-001411.pkl --data=/home/ubuntu/WS-YG/PMG-Net/datasets/Synth_invivo_disp.npy
# Test saved disparity file by sgans for invivo
%run stylegan3/GNet_train.py --dataset=invivo --mode=test_disp